In [1]:
%matplotlib inline

import numpy as np
from PIL import Image, ImageDraw, ImageColor
import math
import matplotlib.pyplot as plt
import onnxruntime as rt
import time
import importlib
import os
from tqdm import notebook
DETRAC_TEST = '/home/ubuntu/ibrahim/master/datasets/Insight-MVT_Annotation_Test'
th = 0.5

In [2]:
os.chdir(os.path.abspath('/home/ubuntu/ibrahim/master/models/research/'))

## SORT

In [3]:
models = [("retina_sort","/home/ubuntu/ibrahim/master/training/ssd_resnet/base/saved_model/")]
to_path=["/home/ubuntu/ibrahim/master/results/","/MOT17-train/data/data/"]

In [4]:
import research.object_detection.tracking.sort.sort
from research.object_detection.tracking.sort.sort import Sort

model_name, model_path = models[0]
path_write = to_path[0]+ model_name+ to_path[1]
if not os.path.exists(path_write):
    os.makedirs(path_write)
sess = rt.InferenceSession(os.path.join(model_path, "model.onnx"),  providers=['CUDAExecutionProvider'])
outputs = ["num_detections", "detection_boxes", "detection_scores", "detection_classes"]

for directory in notebook.tqdm(sorted(os.listdir(DETRAC_TEST))):
    elapsed = []
    if os.path.isdir(os.path.join(DETRAC_TEST, directory)):
        importlib.reload(research.object_detection.tracking.sort.sort)
        mot_tracker= Sort()
        prediticon_file = open(os.path.join(path_write, directory + ".txt"), "w")
        prediticon_np = open(os.path.join(path_write, directory + ".npy"), "wb")
        for image in notebook.tqdm(sorted(os.listdir(os.path.join(DETRAC_TEST, directory)))):
            image_path = os.path.join(DETRAC_TEST, directory, image)
            frame_no = int(image.replace('img', '').replace('.jpg', ''))
            img = Image.open(image_path)
            image_np = np.asarray(img).reshape(img.size[1], img.size[0], 3)
            height, width, _ = image_np.shape
            input_tensor = np.expand_dims(image_np.astype(np.uint8), axis=0)
            start_time = time.time()
            result = sess.run(outputs, {"input_tensor": input_tensor})
            end_time = time.time()
            elapsed.append(end_time - start_time)
            num_detections, boxes, scores, classes = result[0][0], result[1][0], result[2][0], result[3][0]
            det = []
            for f in range(boxes.shape[0]):
                box = boxes[f]
                if scores[f] > th:
                    box_top = int(box[0] * height)
                    box_left = int(box[1] * width)
                    box_bottom = int(box[2] * height)
                    box_right = int(box[3] * width)
                    det.append([box_left, box_top, box_right, box_bottom, scores[f], classes[f]])
            np.save(prediticon_np, det)
            if det:
                track_bbs_ids = mot_tracker.update(np.array(det))
            else:
                track_bbs_ids = mot_tracker.update()
            for track_pred in track_bbs_ids:
                bb_w = track_pred[2] - track_pred[0]
                bb_h = track_pred[3] - track_pred[1]
                prediticon_file.write(f'{frame_no}, {int(track_pred[4])}, {track_pred[0]:.2f}, {track_pred[1]:.2f}, {bb_w:.2f}, {bb_h:.2f}, 1, -1, -1, -1 \n')
        prediticon_file.close()
        prediticon_np.close()
        mean_elapsed = sum(elapsed) / float(len(elapsed))
        print('Elapsed time: ' +directory+" : "+ str(mean_elapsed*1000) + ' millisecond per image')
        print("FPS: " +directory+" : ", 1.0 / mean_elapsed)

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/1470 [00:00<?, ?it/s]

Elapsed time: MVI_39031 : 232.4742485876797 millisecond per image
FPS: MVI_39031 :  4.301551703361421


  0%|          | 0/1120 [00:00<?, ?it/s]

Elapsed time: MVI_39051 : 235.6701563511576 millisecond per image
FPS: MVI_39051 :  4.2432186386381545


...

Elapsed time: MVI_40903 : 228.72521067565344 millisecond per image
FPS: MVI_40903 :  4.37205849344724


  0%|          | 0/1270 [00:00<?, ?it/s]

Elapsed time: MVI_40904 : 228.85465096300982 millisecond per image
FPS: MVI_40904 :  4.369585655314612


  0%|          | 0/1710 [00:00<?, ?it/s]

Elapsed time: MVI_40905 : 227.37476574747186 millisecond per image
FPS: MVI_40905 :  4.398025421653981


## DeepSORT

In [5]:
models = [("retina_deepsort","/home/ubuntu/ibrahim/master/training/ssd_resnet/base/saved_model/")]
to_path=["/home/ubuntu/ibrahim/master/results/","/MOT17-train/data/data/"]

In [6]:
from research.object_detection.tracking.deep_sort.deep_sort import nn_matching
from research.object_detection.tracking.deep_sort.deep_sort.detection import Detection
import research.object_detection.tracking.deep_sort.deep_sort.tracker
from research.object_detection.tracking.deep_sort.deep_sort.tracker import Tracker
from research.object_detection.tracking.deep_sort.tools import generate_detections as gdet
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
model_filename= '/home/ubuntu/ibrahim/master/deep_sort/cosine_metric_learning/output/detrac/cosine-softmax/detrac.pb'
#model_filename= '/home/ubuntu/ibrahim/master/models/research/object_detection/tracking/deep_sort/mars-small128.pb'
encoder = gdet.create_box_encoder(model_filename,batch_size=1)

2022-02-21 06:07:32.202568: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-21 06:07:32.672358: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8822 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:02:00.0, compute capability: 6.1


In [7]:
model_name, model_path = models[0]
path_write = to_path[0]+ model_name+ to_path[1]
if not os.path.exists(path_write):
    os.makedirs(path_write)
sess = rt.InferenceSession(os.path.join(model_path, "model.onnx"), providers=['CUDAExecutionProvider'])
outputs = ["num_detections", "detection_boxes", "detection_scores", "detection_classes"]

for directory in notebook.tqdm(sorted(os.listdir(DETRAC_TEST))):
    elapsed = []
    if os.path.isdir(os.path.join(DETRAC_TEST, directory)):
        importlib.reload(research.object_detection.tracking.deep_sort.deep_sort.tracker)
        deep_tracker = Tracker(nn_matching.NearestNeighborDistanceMetric("cosine", 0.3, None))
        prediticon_file = open(os.path.join(path_write, directory + ".txt"), "w")
        prediticon_np = open(os.path.join(path_write, directory + ".npy"), "wb")
        for image in notebook.tqdm(sorted(os.listdir(os.path.join(DETRAC_TEST, directory)))):
            image_path = os.path.join(DETRAC_TEST, directory, image)
            frame_no = int(image.replace('img', '').replace('.jpg', ''))
            img = Image.open(image_path)
            image_np = np.asarray(img).reshape(img.size[1], img.size[0], 3).astype(np.uint8)
            height, width, _ = image_np.shape
            input_tensor = np.expand_dims(image_np, axis=0)
            start_time = time.time()
            result = sess.run(outputs, {"input_tensor": input_tensor})
            end_time = time.time()
            elapsed.append(end_time - start_time)
            num_detections, boxes, scores, classes = result[0][0], result[1][0], result[2][0], result[3][0]
            det = []
            for f in range(boxes.shape[0]):
                box = boxes[f]
                if scores[f] > th:
                    box_top = int(box[0] * height)
                    box_left = int(box[1] * width)
                    box_bottom = int(box[2] * height)
                    box_right = int(box[3] * width)
                    det.append([box_left, box_top, (box_right-box_left), (box_bottom-box_top), scores[f], classes[f]])
            if det:
                 features = encoder(image_np, np.array(det)[:, 0:4])
            else:
                features = encoder(image_np, np.array(det))
            detections = [Detection((bbox[0], bbox[1], bbox[2], bbox[3]), bbox[4], feature)
                                    for bbox, feature in zip(det, features)]
            np.save(prediticon_np, detections)
            deep_tracker.predict()
            deep_tracker.update(detections)
            track_bbs_ids = []
            for track in deep_tracker.tracks:
                if not track.is_confirmed() or track.time_since_update > 1:
                    continue
                track_bbs_ids.append([*track.to_tlbr(), track.track_id])
            for track_pred in track_bbs_ids:
                bb_w = track_pred[2] - track_pred[0]
                bb_h = track_pred[3] - track_pred[1]
                prediticon_file.write(f'{frame_no}, {int(track_pred[4])}, {track_pred[0]:.2f}, {track_pred[1]:.2f}, {bb_w:.2f}, {bb_h:.2f}, 1, -1, -1, -1 \n')
        prediticon_file.close()
        prediticon_np.close()
        mean_elapsed = sum(elapsed) / float(len(elapsed))
        print('Elapsed time: ' +directory+" : "+ str(mean_elapsed*1000) + ' millisecond per image')
        print("FPS: " +directory+" : ", 1.0 / mean_elapsed)              

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/1470 [00:00<?, ?it/s]

2022-02-21 06:07:34.450432: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8302
2022-02-21 06:07:34.530317: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 11.0.221, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


Elapsed time: MVI_39031 : 229.62772732689268 millisecond per image
FPS: MVI_39031 :  4.3548747864251744


  0%|          | 0/1120 [00:00<?, ?it/s]

...

Elapsed time: MVI_40903 : 227.50189574259633 millisecond per image
FPS: MVI_40903 :  4.395567767626145


  0%|          | 0/1270 [00:00<?, ?it/s]

Elapsed time: MVI_40904 : 231.29608706226497 millisecond per image
FPS: MVI_40904 :  4.323462678081535


  0%|          | 0/1710 [00:00<?, ?it/s]

Elapsed time: MVI_40905 : 233.40848523970934 millisecond per image
FPS: MVI_40905 :  4.284334389013343


## PAE

In [4]:
#models = [("ssd_final_embeddings","/home/ubuntu/ibrahim/master/training/ssd_mobilenet/final_embeddings/saved_model/")]
#models = [("ssd_base_embeddings","/home/ubuntu/ibrahim/master/training/ssd_mobilenet/base_embeddings/saved_model/")]
models = [("retina_final_embeddings","/home/ubuntu/ibrahim/master/training/ssd_resnet/final_embeddings/saved_model/")]
#models = [("ssd_final_embeddings_skip_training","/home/ubuntu/ibrahim/master/training/ssd_mobilenet/final_embeddings_skip/saved_model/")]
to_path=["/home/ubuntu/ibrahim/master/results/","/MOT17-train/data/data/"]
model_name, model_path = models[0]
path_write = to_path[0]+ model_name+ to_path[1]
if not os.path.exists(path_write):
    os.makedirs(path_write)
sess = rt.InferenceSession(os.path.join(model_path, "model.onnx"), providers=['CUDAExecutionProvider'])
outputs = ["num_detections", "detection_boxes", "detection_scores", "detection_classes", "nms_embedding"]

for directory in notebook.tqdm(sorted(os.listdir(DETRAC_TEST))):
    elapsed = []
    if os.path.isdir(os.path.join(DETRAC_TEST, directory)):
        importlib.reload(research.object_detection.tracking.deep_sort.deep_sort.tracker)
        deep_tracker = Tracker(nn_matching.NearestNeighborDistanceMetric("cosine", 0.3, None))
        prediticon_file = open(os.path.join(path_write, directory + ".txt"), "w")
        prediticon_np = open(os.path.join(path_write, directory + ".npy"), "wb")
        for image in notebook.tqdm(sorted(os.listdir(os.path.join(DETRAC_TEST, directory)))):
            image_path = os.path.join(DETRAC_TEST, directory, image)
            frame_no = int(image.replace('img', '').replace('.jpg', ''))
            img = Image.open(image_path)
            image_np = np.asarray(img).reshape(img.size[1], img.size[0], 3).astype(np.uint8)
            height, width, _ = image_np.shape
            input_tensor = np.expand_dims(image_np, axis=0)
            start_time = time.time()
            result = sess.run(outputs, {"input_tensor": input_tensor})
            end_time = time.time()
            elapsed.append(end_time - start_time)
            num_detections, boxes, scores, classes, emb = result[0][0], result[1][0], result[2][0], result[3][0], result[4][0]
            det = []
            features = []
            for f in range(boxes.shape[0]):
                box = boxes[f]
                if scores[f] > th:
                    box_top = int(box[0] * height)
                    box_left = int(box[1] * width)
                    box_bottom = int(box[2] * height)
                    box_right = int(box[3] * width)
                    det.append([box_left, box_top, (box_right-box_left), (box_bottom-box_top), scores[f], classes[f]])
                    features.append(emb[f])
           
            detections = [Detection((bbox[0], bbox[1], bbox[2], bbox[3]), bbox[4], feature)
                                    for bbox, feature in zip(det, features)]
            np.save(prediticon_np, detections)
            deep_tracker.predict()
            deep_tracker.update(detections)
            track_bbs_ids = []
            for track in deep_tracker.tracks:
                if not track.is_confirmed() or track.time_since_update > 1:
                    continue
                track_bbs_ids.append([*track.to_tlbr(), track.track_id])
            for track_pred in track_bbs_ids:
                bb_w = track_pred[2] - track_pred[0]
                bb_h = track_pred[3] - track_pred[1]
                prediticon_file.write(f'{frame_no}, {int(track_pred[4])}, {track_pred[0]:.2f}, {track_pred[1]:.2f}, {bb_w:.2f}, {bb_h:.2f}, 1, -1, -1, -1 \n')
        prediticon_file.close()
        prediticon_np.close()
        mean_elapsed = sum(elapsed) / float(len(elapsed))
        print('Elapsed time: ' +directory+" : "+ str(mean_elapsed*1000) + ' millisecond per image')
        print("FPS: " +directory+" : ", 1.0 / mean_elapsed)

  0%|          | 0/40 [00:00<?, ?it/s]

  0%|          | 0/1470 [00:00<?, ?it/s]

Elapsed time: MVI_39031 : 253.7713817998666 millisecond per image
FPS: MVI_39031 :  3.940554655562528


In [3]:
import onnxruntime as rt
print( rt.get_device()  )
rt.get_available_providers()


GPU


['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']

In [6]:
!pip install onnxruntime-gpu

  Using cached onnxruntime_gpu-1.10.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (104.8 MB)
You should consider upgrading via the '/home/ubuntu/ibrahim/master/models/env/bin/python3.7 -m pip install --upgrade pip' command.


## HyperPram

In [13]:
model_name="retina_final_embeddings_optuna"
to_path=["/home/ubuntu/ibrahim/master/results/","/MOT17-train/data/data/"]
npy_path = to_path[0]+ "retina_final_embeddings"+ to_path[1]
path_write = to_path[0]+ model_name+ to_path[1]
if not os.path.exists(path_write):
    os.makedirs(path_write)
for directory in notebook.tqdm(sorted(os.listdir(npy_path))):
    if directory.endswith(".npy"):
        importlib.reload(research.object_detection.tracking.deep_sort.deep_sort.tracker)
        matching = nn_matching.NearestNeighborDistanceMetric("cosine", 0.77, 4)
        deep_tracker = Tracker(matching, 0.74, 26, 5, 11, 241)
        prediticon_file = open(os.path.join(path_write, directory[:-3] + "txt"), "w")
        with open(npy_path+directory,  "rb") as prediticon_np:
            i=1
            while True:
                try:
                    detections = np.load(prediticon_np, allow_pickle=True)
                except:
                    break
                deep_tracker.predict()
                deep_tracker.update(detections)
                track_bbs_ids = []
                for track in deep_tracker.tracks:
                    if not track.is_confirmed() or track.time_since_update > 1:
                        continue
                    track_bbs_ids.append([*track.to_tlbr(), track.track_id])
                for track_pred in track_bbs_ids:
                    bb_w = track_pred[2] - track_pred[0]
                    bb_h = track_pred[3] - track_pred[1]
                    prediticon_file.write(f'{i}, {int(track_pred[4])}, {track_pred[0]:.2f}, {track_pred[1]:.2f}, {bb_w:.2f}, {bb_h:.2f}, 1, -1, -1, -1 \n')
                i+=1
        prediticon_file.close()

  0%|          | 0/80 [00:00<?, ?it/s]